# Mono Calibration and EXAFS Results

This notebook explores the effect of energy miscalibration on EXAFS results. Careful energy calibration is needed to compare EXAFS measurements, especially those taken at different instruments or even at different times with the same instrument.

EXAFS measurements are typically made using a double-crystal monochromator at a synchrotron beamline.  While other approaches can be used to good quality spectra, the energy calibration methods of other instruments may depend on details of the instruments.  Energy calibration procedures for double-crystal monochromators are pretty uniform, and is the focus of this discussion.   

A double-crystal monochromator uses Bragg's Law to relate the monochromator angle $\theta$ to X-ray energy $E$ with

  $\lambda = \frac{hc}{E} = 2d\sin(\theta) $

With $h$ being Planck's constant, $c$ the speed of light, and $d$ the lattice spacing of the monochromator crystal.  With $d$ in Angstroms and $E$ in eV, $hc \approx 12398.419$ eV A.  The crystal $d$ values for Si, Ge, and C (the most common crystals used) are typically very well known, but there may be thermally induced changes as synchrotron beamlines.  The monochromator usually has a high-quality and well-calibrated (or even encoded) rotation stage, so that the meausure of *relative angle* is assumed to have extremely small errors.  This is not always the case (or wasn't always the case in the past) and effects of run-out and wobble may need to be considered for some cases.  The  crystal is usually cut to a plane of low-order Miller indices with no allowed 2nd harmonic.  The (111), (220), and (311) of silicon and diamond make up the vast majority of monochromator crystals used for EXAFS.  The crystals are mounted to that the face is on the rotation axis to very high precision and the principle and that the rotaion corresponds exactly to $\theta$, with yaw and roll angles being very small.  With careful mount, the values that control the Energy given by a value $\theta$ of the rotation axis are: a) the value of $d$, and b) an angular offset $\theta_0$, where the diffratction planes are parallel to the incident beam.  With a high-quality monochromator as at most synchrotron beamlines, these two values should be sufficient to adjust to calibrate a monochromator. With only two unknowns, it is common to scan across known (or at least reproducible) energies, say the $K$-edges of metal foils, and adjust values of $d$ and/or $\theta_0$ until the energy for all edges is correct.  

Most commonly, the XANES of some metal foil is measured and the maximum of the first derivative is set to the tabulated edge energy for the metal.  This method will give a bit of a variation as the energy resolution changes. These days, most good XAFS beamlines have good energy resolution that this is not much of a problem, but it can be a problem for some older data sets. It should also be said that those tabulated values are not always accurate to better than 1 or 2 eV. In fact, rather than use the tabulated values such as at https://xraydb.xrayabsorption.org/, the values from Kraft et al Review of Scientific Instruments 67, p681 (1996): (https://doi.org/10.1063/1.1146657) should be used, as they were carefully measured with a single, very high-resolution and consistently calibrated monochromator.

Ideally, a well-calibrated monochromator would have a single $d$-spacing (and so good thermal stability) and angular offset (and the incident angle of the beam from the source not vary) that stays calibrated across the edges of many edges.  If the energy needs to be recalibrated very frequently or for each edge, then the energy will be drifting between those edges, and the reason for any need to recalibrate should be investigated.  If the beamline is well collimated and the angle of the beam incident on the monochromator is stable, it is certainly possible to have a monochromator set up so that stays calibrated for weeks and over an energy range of 10 keV or more.  This can be more complicated when a collimating (or any other) mirror is placed before the monochromator, as then very high stability of that mirror is also needed.

For any XAS analysis, the energy scale is critical. Because of the variations above in calibrating  energy, small energy shifts between beamlines or even runs at the same beamline but different months are not uncommon. If the shift is relatively small, say 1 to 5 eV at 10 keV, simply adding a constant  energy offset is satisfactory.
 
You may hear people say that poor calibration leads to large errors in EXAFS results or even leads to "the wrong results".  You should know that when people describe scientific measurements and analysis as "Wrong", they are almost certainly wrong. Measurements have precision and accuracy, and science always allows for improvements in measurement and analysis.  

Here, we'll look at these effects. From this, we will conclude that claims that poor calibration leads to large errors in EXAFS results are not accurate.

In [1]:
# We start with some imports of Python functions we will need to read, 
# modify, and fit xafs data:

import time, os, requests
import numpy as np
import scipy.constants as consts

from larch.io import read_ascii
from larch import Group
from larch.fitting import param_group, param, guess
from larch.xafs import (autobk, feffpath, feffit_transform,
                        feffit_dataset, feffit, feffit_report)
from larch.plot.bokeh_xafsplots import plot, multi_plot

PLANCK_HC = 1.e10 * consts.Planck * consts.c / consts.e
RAD2DEG = 180.0/np.pi
DEG2RAD = np.pi/180.0

In [2]:
# then we will some XAFS data (Cu metal) at the Cu K edge, and a Feff.dat file
# so we'll download these to the current directory
cudata_file = '../xafsdata/cu_10k.xmu'
feffdat_file = '../feffit/feff0001.dat'

root_url = 'https://raw.githubusercontent.com/xraypy/xraylarch/master/examples'
current_dir = os.getcwd()
downloads = {}
if not os.path.exists(cudata_file):
    cudata_file = 'cu_10k.xmu'
    downloads[cudata_file] = f'{root_url}/xafsdata/cu_10k.xmu'
if not os.path.exists(feffdat_file):
    feffdat_file = 'feff0001.dat'
    downloads[feffdat_file] = f'{root_url}/feffit/feff0001.dat'

for dest, source_url in downloads.items():
    t0 = time.time()
    req = requests.get(source_url, verify=True, timeout=1.0)
    print(source_url, req)
    if req.status_code == 200:
        full_dest = os.path.join(current_dir, dest)
        with open(full_dest, 'wb') as fh:
            fh.write(req.content)
        print(f'Downloaded {full_dest}: {time.time()-t0:.3f} sec')
        time.sleep(0.25)
time.sleep(0.1)
print("Got data files: ", cudata_file, feffdat_file)

Got data files:  ../xafsdata/cu_10k.xmu ../feffit/feff0001.dat


In [3]:
# now we will read that data file we just downloaded


cudat = read_ascii(cudata_file, labels='energy, mu')

# This data for Cu metal at the Cu K edge  measured 
# with a Si(111) monochromator.  It is a little cryptic, 
# but this data file set says that  hc_2d = 1977.260
hc_2d = 1977.260
# use the current value of HC to get the dspace they would have used
dspace = PLANCK_HC / (2 * hc_2d)
print(f'Using a d spacing {dspace:.6f} Ang')


# now let's calculate the mono angles in degrees and plot that
cudat.theta = RAD2DEG * np.arcsin(PLANCK_HC / (2.0*dspace*cudat.energy))


aplot = plot(cudat.energy, cudat.theta, xlabel='Energy (eV)', ylabel=r"$\theta$", 
         color='red', title='Si(111) at Cu K edge', label='angle', linewidth=2)

aplot.show()


Using a d spacing 3.135253 Ang


Loading BokehJS ...

note from the plot above that the angle is around 10 to 12 degrees, and that Energy and Angle are not perfectly linear.

In [5]:
# let's plot the normalized XAFS data around the edge:

autobk(cudat.energy, cudat.mu, group=cudat, rbkg=1.0, kw=2)
print(f'E0 = {cudat.e0:.3f} eV')
plot(cudat.energy, cudat.norm, xmin=8900, xmax=9500, label='norm',
     xlabel='Energy (eV)', ylabel=r'$$\mu(E)$$')
 

E0 = 8977.580 eV


Loading BokehJS ...

In [9]:
# let's now write a function to do a first shell fit to the 
# XAFS data from a dataset (Group)
def fit_cu_1stshell(dat, full_report=False, with_plot=False):
   
    # extract the EXAFS chi(k)
    autobk(dat.energy, dat.mu, group=dat, rbkg=1.0, kw=2)

    # define fitting parameter group
    pars = param_group(amp    = param(1.0, vary=True),
                       del_e0 = param(0.0, vary=True),
                       sig2   = param(0.0, vary=True),
                       del_r  = guess(0.0, vary=True) )

    # define a Feff Path, give expressions for Path Parameters
    path1 = feffpath(feffdat_file,
                     s02    = 'amp',
                     e0     = 'del_e0',
                     sigma2 = 'sig2',
                     deltar = 'del_r')

    # do the fit
    trans = feffit_transform(kmin=2.5, kmax=15, kw=2, dk=4, window='kaiser',
                            rmin=1.4, rmax=3.0)
    dset = feffit_dataset(data=dat, pathlist=[path1], transform=trans)
    out = feffit(pars, dset)
    
    r1_fit = out.params['del_r'].value + path1.reff
    r1_err = out.params['del_r'].stderr
    print(f'Result:  E0 = {dat.e0:.3f} eV, R= {r1_fit:.4f} +/- {r1_err:.4f} Ang')
    if full_report:
        print(feffit_report(out))
    if with_plot:
        multi_plot([dict(xdata=dset.data.r, ydata=dset.data.chir_mag, label='data',
                           xtitle='R (Ang)', ytitle=r'$$|\chi(R)|$$'),
                    dict(xdata=dset.model.r, ydata=dset.model.chir_mag, label='fit')])
    return out
    
out = fit_cu_1stshell(cudat, full_report=True, with_plot=True)
   

Result:  E0 = 8977.580 eV, R= 2.5494 +/- 0.0017 Ang
=================== FEFFIT RESULTS ====================
[[Statistics]]
  n_function_calls     = 31
  n_variables          = 4
  n_data_points        = 104
  n_independent        = 13.7323954
  chi_square           = 80.5021987
  reduced chi_square   = 8.27157087
  r-factor             = 0.00122781
  Akaike info crit     = 32.2861097
  Bayesian info crit   = 34.7651404
 
[[Parameters]]
  amp                  =  0.9216005 +/- 0.0271218  (init= 1.0000000)
  del_e0               =  5.6772818 +/- 0.3759213  (init= 0.0000000)
  del_r                =  0.0015526 +/- 0.0016816  (init= 0.0000000)
  sig2                 =  0.0035496 +/- 1.8541e-4  (init= 0.0000000)
 
[[Correlations]] (unreported correlations are <  0.100)
  amp, sig2            =  0.924
  del_e0, del_r        =  0.914
  del_r, sig2          =  0.135
  amp, del_r           =  0.131
 
[[Dataset]]
  unique_id            = 'damwhg4w'
  fit space            = 'r'
  r-range          

Loading BokehJS ...

In [10]:
# OK, so we found E0=8977 eV, and can do a fit with this dataset and get 
# R = 2.5494(0017) Ang.

# If we take the angles for this energy array and change the offset, 
# what do we get? 

# we define two functions to shift X-ray energy either by changing the 
# angular offset or the  d-spacing of the monochromator

def energy_shift(energy, dspace, theta_off=0, dspace_off=0):
    theta = RAD2DEG * np.arcsin(PLANCK_HC / (2.0*dspace*energy))
    return PLANCK_HC/(2*(dspace-dspace_off)*np.sin(DEG2RAD*(theta-theta_off)))

def change_calib(dgroup, dspace, theta_off=0, dspace_off=0):
    "make a new data group with an offset in theta and/or dspacing"
    enew  = energy_shift(dgroup.energy, dspace, theta_off=theta_off, 
                         dspace_off=dspace_off)

    dat = Group(energy=enew, mu=dgroup.mu*1.0)
    autobk(dat.energy, dat.mu, group=dat, rbkg=1.0, kw=2)
    print(f'theta_off={theta_off:.4f}, dspace_off={dspace_off:.4f}: E0= {dgroup.e0:.3f} to {dat.e0:.3f}')
    return dat

# and let's see how big a shift of 0.02 degrees is
# For reference, 0.02 degrees ~= 350 microradians, 
# which is about 10x bigger than the Darwin width for Si(111).
new_dat = change_calib(cudat, dspace, theta_off=0.02)

theta_off=0.0200, dspace_off=0.0000: E0= 8977.580 to 8991.481


In [12]:
# That is, an angular error of 0.02 degrees gives an energy shift of 14 eV.
# Let's plot those XANES spectra together

multi_plot([dict(xdata=cudat.energy, ydata=cudat.norm, label='E0=8977.6'),
            dict(xdata=new_dat.energy, ydata=new_dat.norm, label='E0=8991.5',
                 xmin=8950, xmax=9100, xlabel='Energy (eV)', ylabel=r'$$\mu(E)$$')])

# that is a very big energy shift for XANES work.

Loading BokehJS ...

In [13]:
# and now we can fit that shifted data:

out = fit_cu_1stshell(new_dat, full_report=False, with_plot=False)
   
# and we see that the 1st shell R changed from R= 2.5494 to 2.5455 (+/- 0.0017 Ang).
# a shift in R of 0.004 Ang. 
#
# Yes, that shift is outside of the error bars for this very good dataset.  
# Typical uncertainties and expected accuracies of EXAFS results are 0.01 Ang. 

Result:  E0 = 8991.481 eV, R= 2.5455 +/- 0.0017 Ang


In [14]:
# If we had an angular offset that big, how would we know?

# We would change the monochromator energy to another edge.  
# Let's run our `energy_shift` function with different energies:

etest = np.array([3000, 5000, 7000, 9000, 11000, 20000])
enew = energy_shift(etest, dspace, theta_off=0.02)
for e_in, e_out in zip(etest, enew):
    print(f'{e_in:.1f} -> {e_out:.1f} eV')
    

3000.0 -> 3001.2 eV
5000.0 -> 5004.1 eV
7000.0 -> 7008.3 eV
9000.0 -> 9014.0 eV
11000.0 -> 11021.1 eV
20000.0 -> 20070.5 eV


In [15]:
# if we thought a shift of 14 eV at Cu K was big (it is!), then a shift 
# of 70 eV at Mo K edge is enormous.
# For completeness, what if we had a bad value for $d$?
enew = energy_shift(etest, dspace, dspace_off=0.01)
for e_in, e_out in zip(etest, enew):
    print(f'{e_in:.1f} -> {e_out:.1f} eV')

3000.0 -> 3009.6 eV
5000.0 -> 5016.0 eV
7000.0 -> 7022.4 eV
9000.0 -> 9028.8 eV
11000.0 -> 11035.2 eV
20000.0 -> 20064.0 eV


In [20]:
# We can see that an error in angular offset is more pronounced at high energy, 
# or lower angle. On the other hand, a bad value for d-spacing gives a slightly less 
# energy-dependent error (less of a "chromatic aberration").  
#
# When calibrating a monochromator, it is important to use a wide range of 
# energies to set the d-spacing and angular offset.

# But: how far off do you have to be to see real errors in EXAFS results?

e0val = []
delr1 = []

for theta_off in (0.000, 0.001, 0.002, 0.005, 0.010, 0.020, 0.03, 
                  0.04, 0.050, 0.06, 0.07, 0.08, 0.09, 0.10):
    new_dat = change_calib(cudat, dspace, theta_off=theta_off)

    out = fit_cu_1stshell(new_dat, full_report=False, with_plot=False)
    
    e0val.append(new_dat.e0-cudat.e0) 
    delr1.append(out.params['del_r'].value -0.00155)

e0val = np.array(e0val)
delr1 = np.array(delr1)
   
f = plot(e0val, delr1, xlabel='E0 (eV)', ylabel=r'$$\Delta R1$$ (Ang)', label='deltaR')

theta_off=0.0000, dspace_off=0.0000: E0= 8977.580 to 8977.580
Result:  E0 = 8977.580 eV, R= 2.5494 +/- 0.0017 Ang
theta_off=0.0010, dspace_off=0.0000: E0= 8977.580 to 8978.274
Result:  E0 = 8978.274 eV, R= 2.5492 +/- 0.0017 Ang
theta_off=0.0020, dspace_off=0.0000: E0= 8977.580 to 8978.968
Result:  E0 = 8978.968 eV, R= 2.5490 +/- 0.0017 Ang
theta_off=0.0050, dspace_off=0.0000: E0= 8977.580 to 8981.051
Result:  E0 = 8981.051 eV, R= 2.5484 +/- 0.0017 Ang
theta_off=0.0100, dspace_off=0.0000: E0= 8977.580 to 8984.525
Result:  E0 = 8984.525 eV, R= 2.5474 +/- 0.0017 Ang
theta_off=0.0200, dspace_off=0.0000: E0= 8977.580 to 8991.481
Result:  E0 = 8991.481 eV, R= 2.5455 +/- 0.0017 Ang
theta_off=0.0300, dspace_off=0.0000: E0= 8977.580 to 8998.448
Result:  E0 = 8998.448 eV, R= 2.5436 +/- 0.0017 Ang
theta_off=0.0400, dspace_off=0.0000: E0= 8977.580 to 9005.427
Result:  E0 = 9005.427 eV, R= 2.5417 +/- 0.0016 Ang
theta_off=0.0500, dspace_off=0.0000: E0= 8977.580 to 9012.416
Result:  E0 = 9012.416 eV,

Loading BokehJS ...

In [21]:
# To get an error in R of 0.01 Ang, E0 needs to shift by about 35 eV at the Cu K edge.   
# That is more than double the one shown in the Cu XANES plot above.  That is an 
# absurdly large energy shift.  

# If you are expecting to get absolute measures that are accurate to 0.005 Ang,
# then making sure your energy calibration is good to 10 eV is important.  
# Claiming such absolute accuracies is rare in the EXAFS literature, and does 
# require a careful analysis.

# Energy errors or drifts of 5 eV in the 5 to 15 keV range will cause significant 
# problems for XANES work and would need to be addressed.  XANES Analysis is the 
# reason to work toward good energy calibration and reliability.
#
# Errors of 5 eV in the 5 to 15keV range will not impact EXAFS results at all. 
#
# If someone tells you otherwise, point them to this notebook.
